In [5]:
import pandas as pd
from glob import glob
import os

In [6]:
# naming convention of the name txt files for each year of birth
pattern = os.path.join('names', 'yob*.txt')
files = sorted(glob(pattern))

if not files:
    raise FileNotFoundError(f"No files found with pattern: {pattern}")

dfs = []
for path in files:
    # extract yob from file name
    fname = os.path.basename(path)
    try:
        year = int(fname.replace('yob', '').replace('.txt', ''))
    except ValueError:
        # skip files that don't match the expected pattern
        print(f"Skipping unexpected filename: {fname}")
        continue

    # read file
    df = pd.read_csv(path, header=None, names=['name', 'sex', 'count'], dtype={'name': str, 'sex': str, 'count': int})
    # create year column using yob from file names
    df['year'] = year
    dfs.append(df)

# concatenate dbs, ensure data types are correct + consistent
all_names = pd.concat(dfs, ignore_index=True)
all_names['count'] = all_names['count'].astype(int)
all_names['year'] = all_names['year'].astype(int)

# sort by year and popularity
all_names = all_names.sort_values(['year', 'count'], ascending=[True, False]).reset_index(drop=True)

# write to csv
out_path = os.path.join('names', 'all_years.csv')
all_names.to_csv(out_path, index=False)
print(f"Wrote combined CSV with {len(all_names)} rows to: {out_path}")

Wrote combined CSV with 2149477 rows to: names/all_years.csv
